In [ ]:
#===============================================================================================
# Huggingface와 ONNX 런타임을 이용한 동적 양자화 예시
# => 훈련된 bert 모델을 가지고, ONNX 모데로 만든 후, 테스트 하는 예시
# => 분류모델(NLI포함)이면 ORTModelForSequenceClassification 이용
#
#
#
## 참고 : https://huggingface.co/docs/optimum/index
##        https://github.com/huggingface/optimum
##        https://huggingface.co/blog/optimum-inference
##
## 아래 패키지들을 설치해야 함
'''
!pip install datasets
!pip install optimum
!pip install optimum[onnxruntime]
!pip install optimum[onnxruntime-gpu]  #gpu 사용인 경우

'''    
#===============================================================================================

import numpy as np
import pandas as pd
import torch
import os
import torch.nn.functional as F
import sys

from myutils import seed_everything, GPU_info, mlogging
logger = mlogging(loggername="distilbertnlitest", logfilename="distilbertnlitest")
seed_everything(111)

In [ ]:
#============================================================
# 기존 bert 모델을 동적 양자화 시킴
#============================================================

from optimum.onnxruntime import ORTConfig, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# 기존 bert 모델 경로 
model_checkpoint = "./distilbert-0331-TS-nli-0.1-10"

# 동적 양자화인 경우 is_static = False로 해야 함.
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)

# 분류 모델인 경우에는 feature="sequence-classification"
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="sequence-classification")

# ONNX 모델로 만들고 양자화 함
quantizer.export(
    onnx_model_path="model.onnx",   # ONNX 모델 출력 경로
    # onnx 양자화 모델이 생성되는 경로(이름은 model.onnx로 해야함=>그래야 huggingface 함수 이용시 경로만 지정해도 자동으로 불어옴)
    onnx_quantized_model_output_path="./distilbert-TS/model.onnx",  
    quantization_config=qconfig,
)

In [ ]:
#============================================================
# 양자화 모델 불러옴
# => 양자화 모델은 model.eval() 하면 에러남.
#============================================================

from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction, ORTModel, ORTModelForSequenceClassification

vocab_path = "./distilbert-0331-TS-nli-0.1-10"
model_path = "./distilbert-0331-TS-nli-0.1-10"

tokenizer = AutoTokenizer.from_pretrained(vocab_path)
model = ORTModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

print(model)

In [ ]:
'''
#================================================================
# 기존 bert 모델 불러옴
#================================================================
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

vocab_path = "./distilbert-0331-TS-nli-0.1-10"
model_path = "./distilbert-0331-TS-nli-0.1-10"

tokenizer = DistilBertTokenizer.from_pretrained(vocab_path, do_lower_case=False)
model = DistilBertForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.eval()
'''

In [ ]:
# 평가 data loader 생성

from torch.utils.data import DataLoader, RandomSampler

from myutils import ClassificationDataset, KlueNLICorpus, data_collator, KorNLICorpus
#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
#corpus = KlueNLICorpus()
corpus = KorNLICorpus()

# 평가 dataset 생성
#file_fpath = '../../korpora/klue-nli/klue-nli-v1.1_dev.json'
file_fpath = './korpora/xnli.test.ko.tsv'
    
dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('eval_loader_len: {}'.format(len(eval_loader)))

In [ ]:
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F

logger.info(f"=== model: {model_path} ===")
logger.info(f"eval_file : {file_fpath}")
#logger.info(f"num_parameters: {model.num_parameters()}")

# 평가 시작
#model.eval()

total_loss = 0
total_len = 0
total_correct = 0

start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
    # 입력 값 설정
    # =>**distilbert에는 token_type_ids가 없다
    labels = data['labels']
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
 
    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
    with torch.no_grad():
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                       attention_mask=attention_mask,
                       labels=labels)
    
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)

logger.info(f"eval-accuracy: {total_correct / total_len}")
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

In [ ]:
'''
# 모델 추론 테스트 

import torch.nn.functional as F

inputs = tokenizer("날씨가 흐리다", "비가 내린다",return_tensors="pt")
outputs = model(**inputs)

print(outputs)
logits = outputs.logits
print(logits)

pred = torch.argmax(F.softmax(logits), dim=1)
print(pred)

if pred == 0:
    print('참(수반:entailment)')
elif pred == 1:
    print('거짓(모순:contradiction)')
elif pred == 2:
     print('모름(중립:neutral)')
'''

In [ ]:
'''
# 참고
# => huggingface ORTModelForSequenceClassification 를 이용하여, ONNX 모델로 쉽게 저장할수 있다.(*단 여기서 ONNX 모델은 양자화된 모델은 아니고, 형식만 변경한 것임)
# => 양자화 하려면. 위에서 처럼 ORTQuantizer 이용해야 함.

from optimum.onnxruntime import ORTModelForSequenceClassification

model_checkpoint = "./distilbert-0331-TS-nli-0.1-10"

# Load model from hub and export it through the ONNX format 
model = ORTModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=3,
    from_transformers=True
)

# Save the exported model
model.save_pretrained("./onnxfolder")
'''